In [15]:
import pandas as pd
import numpy as np
import string
import collections
%load_ext autotime
from tqdm import tqdm

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 0 ns (started: 2022-01-20 14:02:17 +05:30)


In [33]:
def create_word_list_new(filename='wordle_words.csv'):
    word_df = pd.read_csv(filename, names=['words'])
    for i in range(5):
        word_df['pos_'+str(i)] = word_df['words'].apply(lambda x: x[i])
    print('Number of words: ', word_df.shape[0])
    return word_df

time: 0 ns (started: 2022-01-20 16:09:50 +05:30)


In [17]:
def count_prob(letter,word_df,num):
    counter = word_df['pos_'+str(num)].apply(lambda x: 1 if x.count(letter)==1 else 0)
    return counter.sum()/(len(word_df))

time: 0 ns (started: 2022-01-20 14:02:18 +05:30)


In [18]:
def generate_alphabet_prob(word_df):
    alphabet_string = string.ascii_lowercase
    alphabet_list = list(alphabet_string)
    alphabet_df = pd.DataFrame(alphabet_list, columns=['letters'])

    for i in range(5):
        column_name = 'prob_'+str(i)
        alphabet_df[column_name] = alphabet_df['letters'].apply(lambda x: count_prob(x,word_df,i))
    
    return alphabet_df

time: 0 ns (started: 2022-01-20 14:02:18 +05:30)


In [19]:
def word_score(word, alphabet_df):
    word_dict = {k: v for v, k in enumerate(list(word))}
    word_df = pd.DataFrame({'letters':word_dict.keys(),'position':word_dict.values()})
    word_df = word_df.merge(alphabet_df, how = 'left', on = ['letters'])
    word_df['prob'] = word_df.apply(find_prob, axis =1)
    return word_df['prob'].sum()

def find_prob(row):
    column_name = 'prob_' + str(row['position'])
    return row[column_name]

time: 0 ns (started: 2022-01-20 14:02:19 +05:30)


In [20]:
def suggest_word(input_word='',round_result='',word_df='',iteration=1,all_suggested=[]):
    if (iteration ==1) & (len(word_df)==0):
        word_letter_df = create_word_list_new()
        alphabet_df = generate_alphabet_prob(word_letter_df)
        word_letter_df['word_prob'] = word_letter_df['words'].apply(lambda x: word_score(x, alphabet_df))
    
    elif (iteration ==1) & (len(word_df)!=0):
        word_letter_df = word_df.copy()

    else:
        word_list = list(input_word)
        result_list = list(round_result)

        black_letters_idx = [idx for idx,letter in enumerate(result_list) if letter =='b']
        black_letters = [word_list[idx] for idx in black_letters_idx]

        green_letters_idx = [idx for idx,letter in enumerate(result_list) if letter =='g']
        green_letters = [word_list[idx] for idx in green_letters_idx]

        yellow_letters_idx = [idx for idx,letter in enumerate(result_list) if letter =='y']
        yellow_letters = [word_list[idx] for idx in yellow_letters_idx]
        
        overlapping_letters_g = intersection(black_letters,green_letters)
        overlapping_letters_y = intersection(black_letters,yellow_letters)
        overlapping_letters_g = list(set(overlapping_letters_g) - set(overlapping_letters_y))
        
        remove_positions_g = dict()
        for i in overlapping_letters_g:
            indices = [j for j, x in enumerate(green_letters) if x == i]
            remove_positions_g[i] = list(set([0,1,2,3,4]) - set([elem for idx,elem in enumerate(green_letters_idx) if idx in indices]))
    
        
        word_letter_df = word_df.copy()
        
        for i in black_letters_idx:
            if word_list[i] in remove_positions_g.keys():
                for j in remove_positions_g[word_list[i]]:
                    word_letter_df = word_letter_df[word_letter_df['pos_'+str(j)]!=word_list[i]]
            elif word_list[i] in overlapping_letters_y:
                word_letter_df = word_letter_df[word_letter_df['pos_'+str(i)]!=word_list[i]]
            else:
                word_letter_df = word_letter_df[word_letter_df['words'].apply(lambda x: x.find(word_list[i])<0)]
        
        for i in green_letters_idx:
            word_letter_df = word_letter_df[word_letter_df['pos_'+str(i)]==word_list[i]]
        
        for i in yellow_letters_idx:
            word_letter_df = word_letter_df[(word_letter_df['pos_'+str(i)]!=word_list[i]) & (word_letter_df['words'].apply(lambda x: x.find(word_list[i])>=0))]
        
        alphabet_df = generate_alphabet_prob(word_letter_df)
        word_letter_df['word_prob'] = word_letter_df['words'].apply(lambda x: word_score(x, alphabet_df))
    
    word_letter_df.sort_values(by=['word_prob'],ascending=False, inplace = True)
    word_letter_df.reset_index(drop=True, inplace=True)
    
    suggested_word = word_letter_df['words'].iloc[0]
    if suggested_word in all_suggested:
        suggested_word = word_letter_df['words'].iloc[1]
    
    return suggested_word, word_letter_df
   

time: 15 ms (started: 2022-01-20 14:02:20 +05:30)


In [21]:
def start(actual_word,suggested_word,word_df,iteration,all_suggested):
    if iteration == 1:
        round_result = 'bbbbb'
        suggested_word, word_df = suggest_word(word_df=word_df)
    else:
        round_result = result(actual_word,suggested_word)
        suggested_word, word_df = suggest_word(suggested_word,round_result,word_df,iteration,all_suggested)
    if (round_result == 'ggggg' or suggested_word == actual_word):
        play = 'n'
    else:
        play = 'y'
    return suggested_word,word_df,play

def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

time: 0 ns (started: 2022-01-20 14:02:21 +05:30)


In [42]:
word_df_initial = create_word_list_new('wordle_words_sol.csv')
alphabet_df_initial = generate_alphabet_prob(word_df_initial)
word_df_initial['word_prob'] = word_df_initial['words'].apply(lambda x: word_score(x, alphabet_df_initial))

Number of words:  2315
time: 9.3 s (started: 2022-01-24 01:18:32 +05:30)


In [23]:
def result(actual_word, suggested_word):
    indices = [0,1,2,3,4]
    indices_copy = indices.copy()
    actual_word_dict = dict(zip(indices,list(actual_word)))
    suggested_word_dict = dict(zip(indices,list(suggested_word)))
    initial_result = ['b','b','b','b','b']
    for i in indices_copy:
        if actual_word_dict[i]==suggested_word_dict[i]:
            initial_result[i]='g'
            actual_word_dict.pop(i)
            suggested_word_dict.pop(i)
            indices.remove(i)
    
    indices_copy = indices.copy()
    for i in indices_copy:
        key_list = list(actual_word_dict.keys())
        val_list = list(actual_word_dict.values())
        if suggested_word_dict[i] in val_list:
            position = val_list.index(suggested_word_dict[i])
            index = key_list[position]
            initial_result[i]='y'
            actual_word_dict.pop(index)
            suggested_word_dict.pop(i)
            indices.remove(i)
            
    return "".join(initial_result)

time: 0 ns (started: 2022-01-20 14:03:21 +05:30)


In [43]:
final = dict()
word_list = list(create_word_list_new('wordle_words_sol.csv')['words'].sample(n = 2315, random_state = 1))
for i in tqdm(word_list):
    play = "y"
    if play == "y":
        suggested_word = ''
        word_df = word_df_initial.copy()
        iteration = 1
        all_suggested=[]
    while play=='y':
        actual_word = i
        suggested_word,word_df,play = start(actual_word,suggested_word,word_df,iteration,all_suggested)
        all_suggested.append(suggested_word)
        if play == 'y':
            iteration +=1
        else:
            final[i] = iteration

final_df = pd.DataFrame({'words':final.keys(),'attempts':final.values()})
final_df.to_csv('results_alternate.csv')

  0%|          | 0/2315 [00:00<?, ?it/s]

Number of words:  2315


100%|██████████| 2315/2315 [23:00<00:00,  1.68it/s]

time: 23min (started: 2022-01-24 01:18:46 +05:30)


In [44]:
final_df['attempts'].describe()

count    2315.000000
mean        3.653132
std         0.886484
min         1.000000
25%         3.000000
50%         4.000000
75%         4.000000
max         8.000000
Name: attempts, dtype: float64

time: 16 ms (started: 2022-01-24 01:41:46 +05:30)


In [45]:
final_df[final_df['attempts']<=6].shape[0]/len(final_df)

0.9930885529157667

time: 16 ms (started: 2022-01-24 01:41:46 +05:30)


In [46]:
final_df.head()

,words,attempts
0,nasty,4
1,befit,4
2,first,3
3,tarot,3
4,aside,3


time: 47 ms (started: 2022-01-24 01:41:46 +05:30)
